In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### Wide Residual Net

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [67]:
#Train the model per the settings specified in the original RESNET paper
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# !python train.py --config configs/cifar/wrn.yaml \
#     model.wrn.depth 28 \
#     model.wrn.widening_factor 10 \
#     train.batch_size 128 \
#     train.base_lr 0.1 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10/exp00 \
#     scheduler.epochs 200
pass

In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_5 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_5/exp00 \
    scheduler.epochs 400

[2020-06-20 17:54:34] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_5
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    a

In [112]:
# Resume training with the un-augmented data
# os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
# #!python train.py --config configs/cifar/resnet.yaml \
# !python train.py --config /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/config.yaml \
#     train.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth \
#     dataset.name CIFAR10 \
#     train.base_lr .001 \
#     train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150 \
#     scheduler.epochs 150

#### Set LEARNING RATE based on ending LR
#    train.resume True \

[2020-06-20 14:52:23] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 32
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_typ

In [90]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume400_50/test_results_0050_cifar10

[2020-06-20 14:12:28] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 26.06it/s]
[2020-06-20 14:12:32] __main__ INFO: Elapsed 3.03
[2020-06-20 14:12:32] __main__ INFO: Loss 0.4499 Accuracy 0.8766


In [113]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/test_results_0150_cifar10

[2020-06-20 16:32:43] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.53it/s]
[2020-06-20 16:32:46] __main__ INFO: Elapsed 2.98
[2020-06-20 16:32:46] __main__ INFO: Loss 0.4499 Accuracy 0.8795


In [91]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume400_50/test_results_0050_cifar101

[2020-06-20 14:12:40] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 16.79it/s]
[2020-06-20 14:12:42] __main__ INFO: Elapsed 0.96
[2020-06-20 14:12:42] __main__ INFO: Loss 0.7680 Accuracy 0.7790


In [114]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/test_results_0150_cifar101

[2020-06-20 16:33:19] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00_resume300_150/checkpoint_00150.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.16it/s]
[2020-06-20 16:33:20] __main__ INFO: Elapsed 0.93
[2020-06-20 16:33:20] __main__ INFO: Loss 0.8206 Accuracy 0.7710


In [79]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/test_results_0050_cifar10

[2020-06-20 13:07:47] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 27.61it/s]
[2020-06-20 13:07:51] __main__ INFO: Elapsed 2.86
[2020-06-20 13:07:51] __main__ INFO: Loss 1.1508 Accuracy 0.8148


In [72]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00200.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/test_results_0200_cifar10

[2020-06-20 13:03:59] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00200.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 26.21it/s]
[2020-06-20 13:04:03] __main__ INFO: Elapsed 3.02
[2020-06-20 13:04:03] __main__ INFO: Loss 0.9363 Accuracy 0.8193


In [78]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/test_results_0300_cifar10

[2020-06-20 13:07:35] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:02<00:00, 26.49it/s]
[2020-06-20 13:07:39] __main__ INFO: Elapsed 2.98
[2020-06-20 13:07:39] __main__ INFO: Loss 1.0633 Accuracy 0.8138


In [77]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/test_results_0400_cifar101

[2020-06-20 13:07:23] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 19.67it/s]
[2020-06-20 13:07:24] __main__ INFO: Elapsed 0.82
[2020-06-20 13:07:24] __main__ INFO: Loss 2.0591 Accuracy 0.6780


In [76]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/test_results_0300_cifar101

[2020-06-20 13:07:14] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/exp00/checkpoint_00300.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 17.68it/s]
[2020-06-20 13:07:15] __main__ INFO: Elapsed 0.91
[2020-06-20 13:07:15] __main__ INFO: Loss 1.9379 Accuracy 0.6705


In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnet_basic_32', 'resnet_basic_32', 'resnet_basic_32']
           'Testset': ['cifar10', 'cifar10', 'cifar10.1']
           'Loss': [0.3604, 0.4011, 0.8051],
           'Epoch': [100, 160, 160],
           'Accuracy': [0.9170, 0.9232, 0.8320],
           'Original_Accuracy': [92.5, 92.5, 84.9],
           'Original_CI': [(92.0, 93.0), (92.0, 93.0), (83.2, 86.4)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32,cifar10,100,0.3604,0.9170,92.5,"(92.0, 93.0)"
1,resnet_basic_32,cifar10,160,0.4011,0.9232,92.5,"(92.0, 93.0)"
2,resnet_basic_32,cifar10.1,160,0.8051,0.8320,84.9,"(83.2, 86.4)"


In [117]:
import pandas as pd
a = pd.Series(['resnet_basic_32_ra_2_5', 400, 'cifar10', 1.1508, 0.8148])
b = pd.Series(['resnet_basic_32_ra_2_5', 300, 'cifar10', 1.0633, 0.8138])
c = pd.Series(['resnet_basic_32_ra_2_5', 400, 'cifar10.1', 2.0591, 0.6780])
d = pd.Series(['resnet_basic_32_ra_2_5', 300, 'cifar10.1',  1.9379, 0.6705])


e = pd.Series(['resnet_basic_32_ra_2_5_refined400', 50, 'cifar10.1', 0.7680, 0.7790])
f = pd.Series(['resnet_basic_32_ra_2_5_refined400', 50, 'cifar10', 0.4499,0.8766])
g = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10', 0.4499, 0.8795])
h = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,b,c,d,e,f, g, h], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 92.5 if row[2] == 'cifar10' else 84.9), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (92.0, 93.0) if row[2] == 'cifar10' else (83.2, 86.4)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_32_ra_2_5,400,cifar10,1.1508,0.8148,92.5,"(92.0, 93.0)"
1,resnet_basic_32_ra_2_5,300,cifar10,1.0633,0.8138,92.5,"(92.0, 93.0)"
2,resnet_basic_32_ra_2_5,400,cifar10.1,2.0591,0.678,84.9,"(83.2, 86.4)"
3,resnet_basic_32_ra_2_5,300,cifar10.1,1.9379,0.6705,84.9,"(83.2, 86.4)"
4,resnet_basic_32_ra_2_5_refined400,50,cifar10.1,0.768,0.779,84.9,"(83.2, 86.4)"
5,resnet_basic_32_ra_2_5_refined400,50,cifar10,0.4499,0.8766,92.5,"(92.0, 93.0)"
6,resnet_basic_32_ra_2_5_refined300,150,cifar10,0.4499,0.8795,92.5,"(92.0, 93.0)"
7,resnet_basic_32_ra_2_5_refined300,150,cifar10.1,0.8206,0.771,84.9,"(83.2, 86.4)"


In [8]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_32/exp00/test_results_0160/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[ -7.153804  ,  -0.1832159 ,  -0.69570637, ...,  -0.50926757,
         -5.526208  , -12.987257  ],
       [  2.862379  ,   7.963458  ,  -6.603018  , ...,  -4.740323  ,
         25.90399   ,  -0.52988565],
       [  4.25749   ,   8.408992  ,  -4.3299227 , ...,  -2.3715498 ,
         13.468082  ,   4.5792727 ],
       ...,
       [ -4.7270765 ,  -1.2400844 ,   1.3852903 , ...,  -0.51062894,
         -3.399443  ,  -2.4969094 ],
       [ -2.7640457 ,  14.635863  ,   6.7449965 , ...,  -1.3011913 ,
         -3.036379  ,  -7.061736  ],
       [ -2.6933427 ,   1.8961854 ,  -3.6396854 , ...,  18.63456   ,
         -2.7524152 ,  -3.2204888 ]], dtype=float32)

In [118]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_32_ra_2_5'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_32_ra_2_5'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)